In [ ]:
# blastdbcmd tutorial
#https://blastedbio.blogspot.com/2012/10/my-ids-not-good-enough-for-ncbi-blast.html

import os, sys, pathlib
from collections import OrderedDict
from collections import defaultdict
import pandas as pd
from Bio.Seq import Seq
import os

# Create a new project folder

In [ ]:
project_name = "GeneID"
#os.chdir(r"C:\Users\SagerFish\Dropbox\Git\syntenyZ") # For windows
os.chdir("/Users/eporetsky/Dropbox/Git/syntenyZ")    # For max, linux
pathlib.Path(project_name+"/").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/custom_loci").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/fasta_loci").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/custom_loci").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/custom_CDS").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/custom_CDS/genes").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/custom_CDS/inbreds").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/custom_intergenic").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/custom_intergenic/genes").mkdir(parents=True, exist_ok=True)
pathlib.Path(project_name+"/custom_intergenic/inbreds").mkdir(parents=True, exist_ok=True)

# Create a dictionary with taxa title ID

In [ ]:
# Returns a dictionary of the taxa ID csv based on the blast DB based on a custom csv file
# For each taxa ID returns a dictionaty key:val like this: '1': 'B73', '2': 'W22' ... 
taxa_dict = {}
title2taxa = open("../FASTA/title2taxa.csv","r")
for line in title2taxa:
    line = line.strip().split(",") # strip remove \n from the end
    taxa_dict[line[1]] = line[0]   # add the lines as a new dicrionary element
# I don't know why it started adding wrong UNICODE encoding to B73 ('\ufeffB73')
taxa_dict["1"] = "B73"
taxa_dict

In [ ]:
# BlastDB doesn't retain the origin chromsome numbers so I manually made a conversion list
# returns a dict like this: 'B73', 'Chr10'): 'gnl|BL_ORD_ID|0', ('B73', 'Chr1'): 'gnl|BL_ORD_ID|1' ...

# I don't know why it started adding wrong UNICODE encoding to B73 ('\ufeffB73')
# Had to add the stupid count and manually change the first row
count = 0

identity_dict = {}
chr2identity = open("../FASTA/chr2identity.csv","r")
for line in chr2identity:
    if count==0:
        identity_dict[("title","chr",)] = "identity" # add values to tuple keys
    else:
        line = line.strip().split(",")            # strip remove \n from the end
        identity_dict[tuple(line[0:2])] = line[2] # add values to tuple keys
    count += 1
identity_dict

In [ ]:
# I need to make a function to merge overlapping exons 

In [ ]:
# Make a function to try to predict tandem duplicates

In [ ]:
# Really short sequences don't return a blast result: 6a_CTX_1

# Run blast on specified genomes using the query.fasta in the project file

In [ ]:
#os.chdir(r"C:\Users\SagerFish\Dropbox\Git\FASTA\BLASTdb") # For windows
os.chdir("/Users/eporetsky/Dropbox/Git/FASTA/BLASTdb/")   # For mac, linux

# Before running blastn to get target hits, make sure that you have the query.fasta file in your project folder

print("Changed directory to:", os.getcwd())
# This used to work but one day it stopped working. Now when I run blasn with multiple db files it doesn't report staxid accurately, as in - almost all of the results are the same "1"
#os.system('blastn -db "B73.db W22.db Mo17.db B97.db CML52.db CML103.db CML228.db CML247.db CML277.db CML322.db CML333.db CML69.db EP1.db F7.db HP301.db Il14H.db Ki11.db Ki3.db Ky21.db M162W.db M37W.db Mo18W.db Ms71.db NC350.db NC358.db Oh43.db Oh7B.db P39.db Tx303.db Tzi8.db PI566673.db" -query ../../syntenyZ/'+project_name+'/query.fasta -out ../../syntenyZ/'+project_name+'/blast_results.tsv -num_threads 4 -evalue 1e-4 -outfmt "6 staxid qseqid qstart qend sseqid sstart send"')
for i in range(1,32):
    os.system('blastn -db '+taxa_dict[str(i)]+'.db -query ../../syntenyZ/'+project_name+'/query.fasta -num_threads 4 -evalue 1e-4 -outfmt "6 staxid qseqid qstart qend sseqid sstart send" >> ../../syntenyZ/'+project_name+'/blast_results.tsv')
os.chdir("../../syntenyZ/"+project_name+"/")
print("Changed directory back to:", os.getcwd())

# Before continuing, filter the blast results to contain only wanted rows

In [ ]:
#os.chdir(project_name+"/")
blast_df = pd.read_table("blast_results.tsv", header=None, names=["taxa","ref", "qs","qe","chr","ss","se"])
blast_df['strand'] = blast_df[['ss','se']].apply(lambda x: "+" if x['ss']<x['se'] else "-", axis=1)
#blast_df = blast_df[blast_df["taxa"]==1]
blast_tolist = blast_df.sort_values(['taxa', 'chr', 'ss',  ], ascending=[True, True, True]).values.tolist()

overlap_list = [list(blast_tolist[0])]
gene_copies = {overlap_list[-1][1]:1}
list_len = len(blast_tolist)
for row in range(1,list_len):
    last = overlap_list[-1]
    nrow = list(blast_tolist[row]) # the new row list
    ovl, ovr = (last[5],last[6]) if last[-1]=="+" else (last[6],last[5]) #overlap_left/right
    rol, ror = (nrow[5],nrow[6]) if nrow[-1]=="+" else (nrow[6],nrow[5]) #row_left/right
    #print(blast_tolist[row])
    if nrow[0] != last[0]:
        gene_copies = {nrow[1]:1} # first count of gene_exon copy
    
    if (nrow[0] == last[0]) & (nrow[4] == last[4]) & (nrow[-1] == last[-1]): # if taxa, chrom, strand equal
        #new_name = nrow[1] if ror-rol > ovr-ovl else last[1]  # keep the name of the larger exon
        #nrow[1]
        #if new_name in gene_copies.keys():
        #    gene_copies[new_name] += 1
        #    new_name = new_name.split("_")[0]+"."+str(gene_copies[new_name])+"_"+new_name.split("_")[1]
        #else:
        #    gene_copies[new_name] = 1
        
        #if (rol == ovl) & (ror == ovr): # if the blast result is the same add * to the name
        #    overlap_list[-1][1] = nrow[1] # if the overlap is equal
        if (rol <= ovl) & (ror >= ovr): # if the new blast row spans over then replace with new row completely
            overlap_list[-1] = nrow # if the new row spans previous, replace
        elif (rol < ovl) & (ror >= ovl) & (ror <= ovr):        
            overlap_list[-1][1] = nrow[1]
            if nrow[-1] == "+":
                overlap_list[-1][5] = rol
            else:
                overlap_list[-1][6] = rol
        
        elif (ror > ovr) & (rol >= ovl) & (rol <= ovr):
            overlap_list[-1][1] = nrow[1]
            if nrow[-1] == "+":
                overlap_list[-1][6] = ror
            else:
                overlap_list[-1][5] = ror
        elif (rol >= ovl) & (ror<=ovr): # if new row is within previous row then skip it
            continue
        else:
            overlap_list.append(nrow)
    else:
        overlap_list.append(nrow)

with open("raw_overlap_list.tsv", "w") as overlap_list_out: 
    for row in overlap_list:
        overlap_list_out.write("\t".join(str(e) for e in row)+"\n")
        print(row)

# The following is optional: Add upstream or downstream DNA padding to all

In [ ]:
left_padding = 0
right_padding = 0

# Generate a list of coordinates using relative start site

In [ ]:
# Assumed that sorted by 6th column first and then by 1st
# the left- and right-most coordinates for each inbred
# 'B73': ['chr10', 72947519, 73642747],
loci_coord = {} 

# the left- and right-most coordinates for each gene
#'B73': [['PROPEP4', 1, 605], ['PROPEP4b*', 87931, 87980],
xlim_coord = {} 

# the coordinates for each exon, further edited to remove overlaps
# 'B73': [['PROPEP4', 'exon1', 1, 605, '-'], ['PROPEP4b*', 'exon1', 87931, 87980, '-'],
gene_coord = {} 

overlap_list_copy = list(overlap_list.copy())

### If I run this twice in a row it overwrites overlap_list and I don't know why ###
# [[1, 'PROPEP4_exon1', 1, 588, 'chr10', 72948124, 72947519, '-'],
last = overlap_list_copy[0]

longest_exon_dict = {}
for line in overlap_list_copy:
    if line[1] not in longest_exon_dict.keys():
        longest_exon_dict[line[1]] = line[3]
    else:
        longest_exon_dict[line[1]] = max(longest_exon_dict[line[1]], line[3])

# Create a dictionary, set default val=1, and increment if there are gene duplications in locus
copy_dict = defaultdict(lambda: 1)
for line in overlap_list_copy:
    # Read each line of the overlap list
    strand = line[-1]
    left = min(int(line[5]), int(line[6]))
    right = max(int(line[5]), int(line[6]))
    genotype = taxa_dict[str(line[0])]
    chromosome = line[4]
    
    if line[1].split("_")[0] == "GeneID":
        # Sometimes the splice-sites don't match but it's easy to see it in alignment vs. manually finding the mismatch
        if line[2] < 4:
            if strand == "+":
                #print(taxa_dict[str(line[0])], line[1],left)
                left = left - line[2] + 1
                #print(left)
            else:
                right = right + line[2] - 1
    # Also check splice-site against exon length, if it is 1 or 2 bp shorter than longest one
    
        if  (longest_exon_dict[line[1]] - line[3]) < 91 and (longest_exon_dict[line[1]] - line[3]) > 0:
            # In this specific instance the stop site in some lines was not within the reference gene range
            padding = 0
            if line[1] == 'GeneID_exon7': 
                padding = 0
            if strand == "+":
                right = right + longest_exon_dict[line[1]] - line[3] + padding
            else:
                left = left - (longest_exon_dict[line[1]] - line[3]) - padding
        
    if len(line[1].split("_")) == 2:
        # Get the gene ID and exon number based on query
        gene = line[1].split("_")[0].split(".")[0]
        exon = line[1].split("_")[1]
    else:
        # Need to check if overlapped blast-hits are renamed to this format
        gene = line[1].split("_")[0] + "_" + line[1].split("_")[1]
        exon = line[1].split("_")[2]
    
    # Because we assume that the overlap list is sorted, first genotype line is upstream
    if genotype not in loci_coord.keys():
        # left_loci is the reference upstream genomic coordinate initated once for each genotype
        left_loci = left - left_padding
        # Start inserting the genomic and relative coordinates to each dictionary
        loci_coord[genotype] = [chromosome, left_loci, right + right_padding]
        xlim_coord[genotype] = [[gene, 1+left_padding, right-left_loci+left_padding]]
        gene_coord[genotype] = [[gene, exon,1+left_padding, right-left_loci+left_padding, strand]]
    else:
        # Every new loci_coord line extend the right loci, because lines are ordered
        loci_coord[genotype][2] = right + right_padding
        # Below assumes the duplicate genes are in order
        if xlim_coord[genotype][-1][0].split(".")[0] == gene: 
            previous_strand = gene_coord[genotype][-1][-1] # I don't think this does anything
            if len(line[1].split("_")) == 2:
                # Need to standarize it, if it has exon written in the exon number or not
                previous_exon = int(gene_coord[genotype][-1][-4].split("exon")[1])
                current_exon = int(exon.split("exon")[1])
            else:
                previous_exon = int(gene_coord[genotype][-1][-4])
                current_exon = int(exon)
            # If exons are consecutive, assume that they are part of the same gene and adjust values
            # if  ((previous_exon+1==current_exon) and strand=="+") or ((previous_exon-1==current_exon) and strand=="-"):
            if  ((previous_exon<current_exon) and strand=="+") or ((previous_exon>current_exon) and strand=="-"):
                xlim_coord[genotype][-1][2] = right-left_loci+left_padding
                gene_coord[genotype][-1].extend([exon, left-left_loci+left_padding, right-left_loci+left_padding, strand])
            # Sometimes minor insertions break exons, so if exon names are the same and arbitrary distance is short keep going
            elif ((left-left_loci-gene_coord[genotype][-1][-2]) < 5000) & (exon==gene_coord[genotype][-1][-4].split("_")[-1]): 
                xlim_coord[genotype][-1][2] = right-left_loci+left_padding
                gene_coord[genotype][-1].extend([exon, left-left_loci+left_padding, right-left_loci+left_padding, strand])
            # Else, assume this is a new duplicate gene
            else: 
                copy_dict[(genotype,gene,)] += 1
                gene = gene+"."+str(copy_dict[(genotype,gene,)])
                xlim_coord[genotype].append([gene, left-left_loci+left_padding, right-left_loci+left_padding])
                gene_coord[genotype].append([gene, exon, left-left_loci+left_padding, right-left_loci+left_padding, strand])            
        else:
            xlim_coord[genotype].append([gene, left-left_loci+left_padding, right-left_loci+left_padding])
            gene_coord[genotype].append([gene, exon, left-left_loci+left_padding, right-left_loci+left_padding, strand])
for key in gene_coord["B73"]:
    print(key[0:5])

In [ ]:
""" You can also manually replace some properties if it's too much work to get to work
"""

gene_coord["CML69"] = [['PROPEP4', 'exon1', 1, 599, '-'], 
['PROPEP4.2', 'exon1', 14203, 14441, '-'], 
['PROPEP4.3', 'exon1', 15211, 15819, '-'],
['PROPEP4.4', 'exon1', 42244, 42370, '-'],
['PROPEP5', 'exon1', 66134, 66580, '-']]

# Extract the genomic sequence for each line. Change `skip = False` to run

In [ ]:
skip = True

with open('fasta_loci/combined.fasta', 'w') as fp: 
    pass
fp.close()
# https://stackoverflow.com/questions/1429556/command-to-get-nth-line-of-stdout

with open('coordinate_list.tsv', 'w') as fp: 
    pass
fp.close()

#count = 0
coordinate_list = open("coordinate_list.tsv",'w')
for genotype, coords in loci_coord.items():
    if skip: break
    chromosome, left, right = coords[0], coords[1], coords[2]
    #print(genotype,chromosome, left, right, identity_dict[(genotype,chromosome)], str(left), str(right))
    #print('blastdbcmd -db ../../FASTA/BLASTdb/'+genotype+'.db -entry "'+identity_dict[(genotype,chromosome)]+'" -range "'+str(left)+'-'+str(right)+'" -strand plus -line_length 1000000000| sed 1d  >> fasta_loci/combined.fasta')
    coordinate_list.write("\t".join([genotype,chromosome,str(left),str(right)])+"\n")
    os.system('echo ">' + genotype + '" >> fasta_loci/combined.fasta')
    os.system('blastdbcmd -db ../../FASTA/BLASTdb/'+genotype+'.db -entry "'+identity_dict[(genotype,chromosome)]+'" -range "'+str(left)+'-'+str(right)+'" -strand plus -line_length 1000000000| sed 1d  >> fasta_loci/combined.fasta')
    #print(os.path.getsize("combined.fasta"))            
    #count += 1
    #if count ==2:
    #    break

In [ ]:
# need to make a custom_loci directory

for genotype, genes in gene_coord.items():
    if os.path.exists("custom_loci/custom_loci_"+genotype+".csv"):
        os.remove("custom_loci/custom_loci_"+genotype+".csv")
    count = 1
    with open("custom_loci/custom_loci_"+genotype+".csv", 'a') as custom_loci:
        custom_loci.write(",name,feature,start,end,strand\n")
        for gene in genes:
            name,exon,left,right,strand = gene[0],gene[1],gene[2],gene[-2],gene[4]
            if name == "PROPEP4":
                name = "PROPEP4.1"
            custom_loci.write(",".join([str(count),name,"mRNA",str(left),str(right),strand])+"\n")
            count+=1
            for exon_count in range(int((len(gene)-1)/4)):
                exon,left,right,strand = gene[1+4*exon_count:5+4*exon_count]
                custom_loci.write(",".join([str(count),exon,"exon",str(left),str(right),strand])+"\n")
                count+=1
                #print(exon_count)
            #print(gene_coord["B73"][2][1:][::4])
    

In [ ]:
with open('assigned_xlims.csv', 'w') as delete: 
    pass
delete.close()

with open('assigned_xlims.csv', 'a') as xlims_out:
    xlims_out.write("genotype,left,right\n")
    for genotype, xlims in xlim_coord.items():
        for xlim in xlims:
            xlims_out.write(genotype+","+str(xlim[1])+","+str(xlim[2])+"\n")

# A fasta parser to save all gene CDS sequences based on coordinates

In [ ]:
CDS_genes = OrderedDict()
CDS_inbreds = {}
add_id_to_inbred_name = True

#Rename variable. The combined.fasta contains all the full DNA seqs for each inbred
# TODO: Implement parsing using biopython SeqIO instead of python text IO parser
# Assumed fasta sequence for each value is on a single line
coordinate_list = open("fasta_loci/combined.fasta",'r')
for row in coordinate_list:
    # Iterate over fasta file and save inbred name and sequence in temps
    if row[0] == ">":
        temp_inbred = row[1:-1] # skips the linebreak
        continue
    else:
        temp_seq = row[:-1] # skips the linebreak
    
    # Open the speicific inbred file
    with open("custom_loci/custom_loci_"+temp_inbred+".csv", 'r') as loci:
        # Prase the custom loci file for each gene
        skip = True
        temp_CDS = '' 
        for loc in loci:
            if skip == True: skip = False ; continue # skip the first line
            loc = loc.strip().split(",")
            # if the line is mRNA then
            if loc[2]=="mRNA":
                # if the temp_CDS is not empty and mRNA means we are done with parsing
                # the previous gene and that we can add the combined CDS to the dictionary
                if temp_CDS != '':
                    # Save the sequence to the  CDS_genes dict file to write later
                    CDS_genes.update({temp_gene:temp_CDS})
                    # If the gene ID is not in dict, start a new key with avaialble
                    if temp_gene not in CDS_inbreds.keys():
                        CDS_inbreds[temp_gene] = OrderedDict({temp_inbred_id:temp_CDS})
                    # Not sure why, or if, this else statement is needed, could add a print command
                    # Maybe it is for reverse-complement sequences?
                    else:
                        CDS_inbreds[temp_gene].update({temp_inbred_id:temp_CDS})
                temp_gene = loc[1]
                # A boolean of wether to add gene ID to inbred ID. One of the outputs is a fasta
                # file of all CDS sequences for each inbred, and easier to handle if it has built-in ID
                # of both gene and inbred. Alternatively, could be save as just inbred name
                if add_id_to_inbred_name:
                    temp_inbred_id = temp_inbred+"_"+temp_gene
                else:
                    temp_inbred_id = temp_inbred
                    
                temp_gene = loc[1]
                temp_CDS = ''
            
            # When iterating over exons (aka not mRNA) write the DNA sequence to CDS seq
            elif loc[-1]=="+":
                if loc[3]=="1": loc[3]=0 # xlims start with 1 instead of 0, but other coordinates correct
                temp_CDS += temp_seq[int(loc[3]):int(loc[4])+1]
            elif loc[-1]=="-":
                if loc[3]=="1": loc[3]=0 # xlims start with 1 instead of 0, but other coordinates correct
                temp_CDS = str(Seq(temp_seq[int(loc[3]):int(loc[4])+1]).reverse_complement()) + temp_CDS
        
        # Since the last line is not mRNA need to manually add the last gene
        # Copied that from above to add the last gene
        CDS_genes.update({temp_gene:temp_CDS})
        if temp_gene not in CDS_inbreds.keys():
            CDS_inbreds[temp_gene] = OrderedDict({temp_inbred_id:temp_CDS})
        else:
            CDS_inbreds[temp_gene].update({temp_inbred_id:temp_CDS})
    
    with open("custom_CDS/inbreds/"+temp_inbred+".fasta", "w") as inbred_CDS:
        for key, val in CDS_genes.items():
            inbred_CDS.write(">"+key+"\n")
            inbred_CDS.write(val+"\n")

# This has to be written after all inbred sequences were collected            
for gene, gene_dict in CDS_inbreds.items():
    with open("custom_CDS/genes/"+gene+".fasta", "w") as genes_CDS:
        for key, val in gene_dict.items():
            genes_CDS.write(">"+key+"\n")
            genes_CDS.write(val+"\n")

# A fasta parser to save all gene integenic sequences based on coordinates

In [ ]:
#os.chdir(project_name+"/")
intergenic_genes = OrderedDict()
intergenic_inbreds = {}

add_id_to_inbred_name = False

# Rename variable. The combined.fasta contains all the full DNA seqs for each inbred
# TODO: Implement parsing using biopython SeqIO instead of python text IO parser
# Assumed fasta sequence for each value is on a single line
coordinate_list = open("fasta_loci/combined.fasta",'r')
for row in coordinate_list:
    # Iterate over fasta file and save inbred name and sequence in temps
    if row[0] == ">":
        temp_inbred = row[1:-1] # skips the linebreak
        continue
    else:
        temp_seq = row[:-1] # skips the linebreak
    
    intergeneic_coordinates_list = []
    intergeneic_genes_list = []
    # Open the speicific inbred file
    with open("custom_loci/custom_loci_"+temp_inbred+".csv", 'r') as loci:
        # Prase the custom loci file for each gene
        skip = True
        temp_CDS = '' 
        for loc in loci:
            if skip == True: skip = False ; continue # skip the first line
            loc = loc.strip().split(",")
            # if the line is mRNA then
            if loc[2]=="mRNA":
                # To get intergenic regions we just get the start end site of each mRNA
                # and append to list. For the time being, assume coordinate [0] is skippable
                intergeneic_coordinates_list.append(int(loc[3]))  # gene start
                intergeneic_coordinates_list.append(int(loc[4]))  # gene end
                intergeneic_genes_list.append(loc[1])             # gene ID
                
        for i in range(1, len(intergeneic_genes_list)):
            # Skipped the first and last coordinate_list to get the start and end of each pair of genes
            temp_intergenic = temp_seq[intergeneic_coordinates_list[i*2-1]+1:intergeneic_coordinates_list[i*2]]
            intergenic_genes.update({intergeneic_genes_list[i]:temp_intergenic})
            # A boolean of wether to add gene ID to inbred ID. One of the outputs is a fasta
            # file of all CDS sequences for each inbred, and easier to handle if it has built-in ID
            # of both gene and inbred. Alternatively, could be save as just inbred name
            
            if add_id_to_inbred_name:
                temp_inbred_id = temp_inbred+"_"+intergeneic_genes_list[i]
            else:
                temp_inbred_id = temp_inbred
            # Once we have the temporary inbred_id either with gene or without, add to dict
            if intergeneic_genes_list[i] not in intergenic_inbreds.keys():
                intergenic_inbreds[intergeneic_genes_list[i]] = OrderedDict({temp_inbred_id:temp_intergenic})
            else:
                intergenic_inbreds[intergeneic_genes_list[i]].update({temp_inbred_id:temp_intergenic})
    
    with open("custom_intergenic/inbreds/"+temp_inbred+".fasta", "w") as inbred_CDS:
        for key, val in intergenic_genes.items():
            inbred_CDS.write(">"+key+"\n")
            inbred_CDS.write(val+"\n")

# This has to be written after all inbred sequences were collected            
for gene, gene_dict in intergenic_inbreds.items():
    with open("custom_intergenic/genes/"+gene+".fasta", "w") as genes_CDS:
        for key, val in gene_dict.items():
            genes_CDS.write(">"+key+"\n")
            genes_CDS.write(val+"\n")

# Combine custom_loci with predicted GRINST TEs

In [ ]:
# os.getcwd()
# os.chdir(project_name+"/")
# pathlib.Path("custom_TE_loci").mkdir(parents=True, exist_ok=True)

In [ ]:
# Since the coordinates for the TEs are based of intergenic starts sites, need to iterate over genes
# Iterate over a file list that is based on the GRINST results. File names based on gene infront of intergenic


In [ ]:
grinst_list = []
for file in os.listdir("custom_TE_loci/"):
    if file.split("_")[1] == "grinst.csv":
        grinst_list.append(file.split("_")[0])
print(grinst_list)
for file in os.listdir("custom_loci/"):
    if file.split(".")[-1] != "csv": continue
    #if file != "custom_loci_M162W.csv": break # just for testing
    temp_inbred = file.split(".")[0].split("_")[-1]
    # This will be the new custom TE loci file to writeline into
    f = open("custom_TE_loci/custom_loci_"+temp_inbred+".csv", "a")
    f.write(",name,feature,start,end,strand\n")
    # Start parsing original custom_loci file for each inbred
    with open("custom_loci/custom_loci_"+temp_inbred+".csv", 'r') as loci:
    # Parse the custom loci file for each gene and keep track of the mRNA end
    # For each mRNA encounter, parse the grinst list and check if matches
    # If grinst matches mRNA, using the end of the previous mRNA end to offset
        
        line_count = 1
        skip = True
        list_gene_id, list_end_coord = [], []
        for loc in loci:
            if skip == True: skip = False ; continue # skip the first line
            loc = loc.strip().split(",")
            if loc[2]=="mRNA":
                list_gene_id.append(loc[1])
                list_end_coord.append(loc[4])
                for grinst in grinst_list:
                    if grinst==list_gene_id[-1]:
                        offset = int(list_end_coord[-2])
                        # Start parsing the grinst TE file
                        with open("custom_TE_loci/"+grinst+"_grinst.csv", 'r') as TEs:
                            skip = True
                            for te in TEs:
                                if skip == True: skip = False ; continue # skip the first line
                                te = te.strip().split(",")
                                if te[0] != temp_inbred: continue
                                ostart, oend = str(int(te[1]) + offset), str(int(te[2]) + offset)
                                write_list = [str(line_count), te[6], "mRNA", ostart, oend, "+"]
                                f.writelines(",".join(write_list)+"\n")
                                line_count += 1
                                write_list = [str(line_count), te[6], "exon", ostart, oend, "+"]
                                f.writelines(",".join(write_list)+"\n") 
                                line_count += 1
            loc[0] = str(line_count)
            f.writelines(",".join(loc)+"\n")
            line_count += 1 # increase line_count by 1 after each writelines everywhere
            print(loc)
    f.close()